In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gdp
from shapely.geometry import Point, Polygon

ImportError: libpoppler.so.71: cannot open shared object file: No such file or directory

In [ ]:
data = pd.read_csv('data/train.csv')

In [ ]:
data.head(1)

In [ ]:
data.ciudad.nunique()

In [ ]:
data.provincia.nunique()

In [ ]:
data.idzona.nunique()

In [ ]:
data.idzona.value_counts().nlargest(10)

In [ ]:
data.lat.nunique()

In [ ]:
data.lng.nunique()

# Pruebo Geopandas

In [ ]:
crs = {'init': 'epsg:4326'} #Indico el tipo de coordenadas a usar para geopandas

In [ ]:
geometry = [Point(xy) for xy in zip(data['lat'].dropna(), data['lng'].dropna())]
len(geometry)

In [ ]:
data[['lat', 'lng']].size

In [ ]:
geo_df = gdp.GeoDataFrame(data.dropna(subset = ['lat', 'lng']), crs = crs, geometry = geometry)
geo_df.head()

In [ ]:
geo_df.columns

In [ ]:
street_map = gdp.read_file(gdp.datasets.get_path('naturalearth_lowres'))
fig, ax = plt.subplots(figsize = (15, 15))
street_map.plot(ax = ax, alpha = 0.4, color = 'blue')
geo_df[geo_df['piscina'] == 1.0].plot(ax = ax, markersize = 20, color = 'red', marker = 'o', label = 'Positivo')
#plt.legend(prop = {'size':15})

In [ ]:
gdp.datasets.available

In [ ]:
df = gdp.read_file(gdp.datasets.get_path('nybb'))
ax = df.plot(figsize=(10, 10), alpha=0.5, edgecolor='k')


In [ ]:
import plotly.graph_objects as go

# Load data frame and tidy it.
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_us_ag_exports.csv')

fig = go.Figure(data=go.Choropleth(
    locations=df['code'], # Spatial coordinates
    z = df['total exports'].astype(float), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Reds',
    colorbar_title = "Millions USD",
))

fig.update_layout(
    title_text = '2011 US Agriculture Exports by State',
    geo_scope='usa', # limite map scope to USA
)

fig.show()



In [ ]:
data.columns

In [ ]:
gdp.datasets

In [ ]:
world = gdp.read_file(gdp.datasets.get_path('naturalearth_lowres'))
cities = gdp.read_file(gdp.datasets.get_path('naturalearth_cities'))
world.plot()
geo_df[geo_df['piscina'] == 1.0].plot(ax = ax, markersize = 20, color = 'red', marker = 'o', label = 'Positivo')
#plt.legend(prop = {'size':15})